In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

class WaterQualityLSTMForecaster:
    def __init__(self, data_path):
        # Load and preprocess the data
        self.df = pd.read_csv(data_path, parse_dates=['Date'])
        self.df.set_index('Date', inplace=True)
        
        # Check for missing values
        print("Missing values:\n", self.df.isnull().sum())
        
        # Fill missing values if any
        self.df.fillna(method='ffill', inplace=True)
        
        # Identify water quality parameters and exogenous variables
        self.quality_params = [
            'Biochemical Oxygen Demand', 'Conductivity', 
            'Turbidity', 'Dissolved Oxygen', 
            'Fecal Coliform', 'Fecal Streptococci', 
            'Nitrate', 'pH', 'Total Coliform'
        ]
        self.exogenous_vars = ['Temperature', 'Rainfall']
        
        # Create models directory
        self.model_dir = 'water_quality_models'
        os.makedirs(self.model_dir, exist_ok=True)
        
        # Preprocessing
        self.scalers = {}
        self.scaled_data = self._preprocess_data()
    
    def _preprocess_data(self):
        # Create copy of dataframe
        data = self.df.copy()
        
        # Scale each parameter separately
        scaled_data = {}
        for column in self.quality_params + self.exogenous_vars:
            scaler = MinMaxScaler()
            scaled_data[column] = scaler.fit_transform(data[[column]])
            self.scalers[column] = scaler
        
        return scaled_data
    
    def _create_sequences(self, data, exogenous, seq_length, forecast_horizon):
        X, y = [], []
        X_exo = []
        
        for i in range(len(data) - seq_length - forecast_horizon + 1):
            # Input sequence
            X.append(data[i:i+seq_length])
            
            # Target (forecast) sequence
            y.append(data[i+seq_length:i+seq_length+forecast_horizon])
            
            # Corresponding exogenous variables for forecast period
            X_exo.append(exogenous[i+seq_length:i+seq_length+forecast_horizon])
        
        return (np.array(X), np.array(y), np.array(X_exo))
    
    def build_multivariate_model(self, seq_length, n_features, exo_features, forecast_horizon):
        # Input for historical time series
        input_series = Input(shape=(seq_length, n_features))
        
        # Input for exogenous variables
        input_exo = Input(shape=(forecast_horizon, exo_features))
        
        # LSTM processing of time series
        x = LSTM(64, return_sequences=True)(input_series)
        x = Dropout(0.2)(x)
        x = LSTM(32, return_sequences=False)(x)
        x = Dropout(0.2)(x)
        
        # Reshape exogenous input
        exo_flat = Flatten()(input_exo)
        
        # Combine time series features with exogenous variables
        merged = Dense(64, activation='relu')(x)
        merged = tf.keras.layers.Concatenate()([merged, exo_flat])
        
        # Output layer
        output = Dense(forecast_horizon, activation='linear')(merged)
        
        # Create model
        model = Model(inputs=[input_series, input_exo], outputs=output)
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
        
        return model
    
    def train_and_forecast(self, seq_length=10, forecast_horizon=5):
        results = {}
        accuracy_metrics = {}

        for param in self.quality_params:
            print(f"\nProcessing {param}")
            # Prepare data for this parameter
            data = self.scaled_data[param]
            exogenous = np.column_stack([self.scaled_data[var] for var in self.exogenous_vars])
            
            # Create sequences
            X, y, X_exo = self._create_sequences(data, exogenous, seq_length, forecast_horizon)
            
            if len(X) == 0:
                print(f"Warning: Not enough data for {param}")
                continue
                
            # Reshape inputs
            X = X.reshape(X.shape[0], X.shape[1], 1)
            y = y.reshape(y.shape[0], y.shape[1])
            
            # Split data
            X_train, X_test, y_train, y_test, X_exo_train, X_exo_test = train_test_split(
                X, y, X_exo, test_size=0.2, random_state=42
            )
            
            # Build model
            model = self.build_multivariate_model(
                seq_length=seq_length, 
                n_features=1, 
                exo_features=len(self.exogenous_vars),
                forecast_horizon=forecast_horizon
            )
            
            # Train model
            model.fit(
                [X_train, X_exo_train], y_train, 
                epochs=100, 
                batch_size=32, 
                validation_split=0.2, 
                verbose=1
            )
            
            # Save model
            model_path = os.path.join(self.model_dir, f'{param.replace(" ", "_")}_water_quality_lstm_model.keras')
            model.save(model_path)
            print(f"Model for {param} saved to {model_path}")
            
            # Make predictions on test set
            y_pred = model.predict([X_test, X_exo_test])
            
            # Make future forecast
            # Get the last sequence of historical data
            last_sequence = X[-1].reshape(1, seq_length, 1)
            # Get the corresponding exogenous variables for the forecast period
            last_exo = X_exo[-1].reshape(1, forecast_horizon, len(self.exogenous_vars))
            
            # Make the forecast
            forecast = model.predict([last_sequence, last_exo])[0]
            
            # Inverse transform
            scaler = self.scalers[param]
            results[param] = {
                'forecast': scaler.inverse_transform(forecast.reshape(-1, 1)).flatten(),
                'historical': scaler.inverse_transform(data[-seq_length:].reshape(-1, 1)).flatten()
            }
            
            # Compute accuracy metrics
            y_test_actual = scaler.inverse_transform(y_test)
            y_pred_actual = scaler.inverse_transform(y_pred)
            
            mae = mean_absolute_error(y_test_actual.flatten(), y_pred_actual.flatten())
            rmse = mean_squared_error(y_test_actual.flatten(), y_pred_actual.flatten(), squared=False)
            r2 = r2_score(y_test_actual.flatten(), y_pred_actual.flatten())
            
            accuracy_metrics[param] = {
                'MAE': mae,
                'RMSE': rmse,
                'R2': r2
            }
        
        # Plotting
        if results:
            self._plot_results(results)
        
        return results, accuracy_metrics
    
    def _plot_results(self, results):
        # Create a subplot for each parameter
        n_params = len(results)
        rows = (n_params + 2) // 3  # Ceiling division
        fig, axes = plt.subplots(rows, 3, figsize=(20, 5*rows))
        axes = axes.ravel()
        
        for i, (param, data) in enumerate(results.items()):
            ax = axes[i]
            historical = data['historical']
            forecast = data['forecast']
            
            # Plot historical and forecast
            x_hist = range(len(historical))
            x_forecast = range(len(historical)-1, len(historical)+len(forecast)-1)
            
            ax.plot(x_hist, historical, label='Historical', color='blue')
            ax.plot(x_forecast, forecast, label='Forecast', color='red', linestyle='--')
            
            ax.set_title(param)
            ax.legend()
        
        # Remove empty subplots if any
        for i in range(i+1, len(axes)):
            fig.delaxes(axes[i])
        
        plt.tight_layout()
        plt.savefig('water_quality_forecast.png')
        plt.close()
    
    def load_parameter_model(self, parameter):
        """
        Load a saved model for a specific parameter
        
        Args:
            parameter (str): Name of the water quality parameter
        
        Returns:
            Loaded Keras model
        """
        model_path = os.path.join(self.model_dir, f'{parameter.replace(" ", "_")}_water_quality_lstm_model.keras')
        return load_model(model_path)


# Usage
if __name__ == "__main__":
    # Path to your CSV file
    data_path = r"C:\Users\hp\OneDrive\Desktop\Ganga_Project\Ganga_Project\Devprayag.csv"
    
    # Create forecaster
    forecaster = WaterQualityLSTMForecaster(data_path)
    
    # Train and forecast
    results, accuracy_metrics = forecaster.train_and_forecast()
    
    # Print forecast results
    for param, data in results.items():
        print(f"\n{param} Forecast:")
        print("Historical:", data['historical'])
        print("Forecast:", data['forecast'])
    
    # Print accuracy metrics
    print("\nModel Accuracy Metrics:")
    for param, metrics in accuracy_metrics.items():
        print(f"\n{param} Accuracy:")
        print(f"Mean Absolute Error (MAE): {metrics['MAE']:.4f}")
        print(f"Root Mean Squared Error (RMSE): {metrics['RMSE']:.4f}")
        print(f"R² Score: {metrics['R2']:.4f}")

Missing values:
 Biochemical Oxygen Demand    0
Conductivity                 0
Temperature                  0
Turbidity                    0
Dissolved Oxygen             0
Fecal Coliform               0
Fecal Streptococci           0
Nitrate                      0
pH                           0
Rainfall                     0
Total Coliform               0
WQI                          0
Quality                      0
dtype: int64

Processing Biochemical Oxygen Demand
Epoch 1/100


C:\Users\hp\AppData\Local\Temp\ipykernel_12928\151681147.py:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.df.fillna(method='ffill', inplace=True)


36/36 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - loss: 0.1285 - val_loss: 0.0154
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0170 - val_loss: 0.0115
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0140 - val_loss: 0.0094
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0124 - val_loss: 0.0088
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0107 - val_loss: 0.0089
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.0102 - val_loss: 0.0083
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0098 - val_loss: 0.0072
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0095 - val_loss: 0.0078
Epoch 9/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 0.0091 - val_loss: 0.0065
Epoch 10/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0077 - val_loss: 0.0062
Epoch 11/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0080 - val_loss: 0.0059
Epoch 12/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0

c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - loss: 0.2156 - val_loss: 0.0216
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0195 - val_loss: 0.0104
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0125 - val_loss: 0.0080
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0109 - val_loss: 0.0074
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0101 - val_loss: 0.0073
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0096 - val_loss: 0.0066
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0093 - val_loss: 0.0065
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0083 - val_loss: 0.0069
Epoch 9/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0080 - val_loss: 0.0061
Epoch 10/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0078 - val_loss: 0.0061
Epoch 11/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0075 - val_loss: 0.0064
Epoch 12/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0

...

36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 0.1809 - val_loss: 0.0105
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0120 - val_loss: 0.0047
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0077 - val_loss: 0.0044
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0066 - val_loss: 0.0036
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0059 - val_loss: 0.0041
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0055 - val_loss: 0.0033
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0051 - val_loss: 0.0032
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0048 - val_loss: 0.0032
Epoch 9/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0042 - val_loss: 0.0026
Epoch 10/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0042 - val_loss: 0.0026
Epoch 11/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0040 - val_loss: 0.0028
Epoch 12/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0

c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 29ms/step - loss: 0.2193 - val_loss: 0.0190
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0207 - val_loss: 0.0097
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0133 - val_loss: 0.0097
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0121 - val_loss: 0.0092
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0117 - val_loss: 0.0108
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0111 - val_loss: 0.0086
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0105 - val_loss: 0.0090
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0096 - val_loss: 0.0085
Epoch 9/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0096 - val_loss: 0.0083
Epoch 10/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0093 - val_loss: 0.0082
Epoch 11/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0086 - val_loss: 0.0085
Epoch 12/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0086

c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



Biochemical Oxygen Demand Forecast:
Historical: [0.97402177 1.02013027 0.98166137 0.90990862 0.92973466 0.88987822
 1.003929   0.92611904 1.04059954 0.90325541]
Forecast: [0.81567335 0.8802472  0.8328299  0.91763633 0.8118095 ]

Conductivity Forecast:
Historical: [178.790107  184.269819  179.5820315 170.8927775 173.2118569 168.3561197
 182.0012746 172.5824577 186.278635  169.7025995]
Forecast: [160.44559 171.34439 164.14998 171.20337 164.37155]

Turbidity Forecast:
Historical: [3.34459822 3.50129625 3.40000671 3.19725122 3.27405344 3.16882252
 3.53333983 3.31246162 3.67845144 3.27605124]
Forecast: [3.0885453 3.3595037 3.2089076 3.4757404 3.1590033]

Dissolved Oxygen Forecast:
Historical: [10.71766508 10.78588045 10.71977514 10.60079164 10.62718131 10.55878153
 10.73471951 10.6060059  10.78257586 10.55929001]
Forecast: [10.452904 10.588308 10.475922 10.611143 10.499552]

Fecal Coliform Forecast:
Historical: [2.22124017 2.23688048 2.22567004 2.20389675 2.21121029 2.19957752
 2.23682328 

In [18]:
import os
print(os.getcwd())


C:\Users\New User
